# Security Course
## Amirkabir University of Technology (Tehran Polytechnic)
<img src="../assets/AKUT-logo.png" alt="Amirkabir University of Technology logo" style="width:400px;height:300px;">

## DES Encryption Algorithm

In [ ]:
def DES(plain_text: str, key: str) -> str:
    pass

In [ ]:
def round_fucntion(R: str, round_key: str) -> str:
    pass